# 卷积神经网络开放题

# 数据集
本次开放题将与课程内容保持一致，将使用图像数据集Fashion-MNIST [1] 进行计算机视觉任务的设计，该数据集由衣服、鞋子等服饰组成，共10个类别。

这里简介将此数据集转换成卷积神经网络所需要的输入格式的方法：

## 加载数据集

首先导入本作业需要的包或模块。

In [8]:
import torchvision
import torch
from matplotlib import pyplot as plt
from IPython import display

通过`load_data_fashion_mnist`函数对数据集进行加载，另外函数还指定了参数`transform = transforms.ToTensor()`使所有数据转换为`Tensor`，如果不进行转换则返回的是PIL图片。`transforms.ToTensor()`将尺寸为 (H x W x C) 且数据位于 [0, 255] 的PIL图片或者数据类型为`np.uint8`的NumPy数组转换为尺寸为 (C x H x W) 且数据类型为`torch.float32`且位于 [0.0, 1.0] 的`Tensor`。

我们将在训练数据集上训练模型，并将训练好的模型在测试数据集上评价模型的表现。函数中`mnist_train`是`torch.utils.data.Dataset`的子类，所以我们可以将其传入`torch.utils.data.DataLoader`来创建一个读取小批量数据样本的`DataLoader`实例。

在实践中，数据读取经常是训练的性能瓶颈，特别当模型较简单或者计算硬件性能较高时。PyTorch的`DataLoader`中一个很方便的功能是允许使用多进程来加速数据读取。这里我们通过参数`num_workers`来设置进程读取数据。

In [2]:
def load_data_fashion_mnist(batch_size, resize=None, root="/home/kesci/input/FashionMNIST2065/"):
    """Download the fashion mnist dataset and then load into memory."""
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())

    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)

    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

    return train_iter, test_iter

对于AlexNet，我们需要将 Fashion-MNIST 数据集的图像高和宽扩大到224，这个可以通过在`load_data_fashion_mnist`中传入`Resize`来实现，这边设置数据的`batch_size`为128。

In [3]:
batch_size = 128
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

Fashion-MNIST中一共包括了10个类别，分别为t-shirt（T恤）、trouser（裤子）、pullover（套衫）、dress（连衣裙）、coat（外套）、sandal（凉鞋）、shirt（衬衫）、sneaker（运动鞋）、bag（包）和ankle boot（短靴）。以下函数可以将数值标签转成相应的文本标签。

In [4]:
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

下面定义一个可以在一行里画出多张图像和对应标签的函数。

In [5]:
def show_fashion_mnist(images, labels):
    """Use svg format to display plot in jupyter"""
    display.set_matplotlib_formats('svg')
    # 这里的_表示我们忽略（不使用）的变量
    _, figs = plt.subplots(1, len(images), figsize=(12, 12))
    for f, img, lbl in zip(figs, images, labels):
        f.imshow(img.view((224, 224)).numpy())
        f.set_title(lbl)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)

读取训练数据集中第一个`batch`的数据。

In [6]:
train_data = iter(train_iter)
images, labels = next(train_data)

现在，我们看一下训练数据集中前10个样本的图像内容和文本标签。

In [7]:
labels = get_fashion_mnist_labels(labels)
show_fashion_mnist(images[:10], labels[:10])
plt.show()

<Figure size 864x864 with 10 Axes>

# 卷积神经网络（CNN）

对于计算机视觉的分类任务，在很长一段时间里流行的是研究者通过经验与智慧所设计并生成的手工特征。这类图像分类研究的主要流程是：

1. 获取图像数据集；
2. 使用已有的特征提取函数生成图像的特征；
3. 使用机器学习模型对图像的特征分类。

`卷积神经网络`（CNN）就是含`卷积层`的神经网络，深度卷积神经网络的兴起改变了计算机视觉任务中手工设计的特征的传统，引领了诸多影响深远的研究。

## LeNet

LeNet [2] 就是交替使用卷积层和最大池化层后接全连接层来进行图像分类。
LeNet 诞生于 1994 年，是最早的卷积神经网络之一，并且推动了深度学习领域的发展。自从 1988 年开始，在许多次成功的迭代后，这项由 Yann LeCun 完成的开拓性成果被命名为 LeNet5。LeNet5 的架构基于这样的观点：（尤其是）图像的特征分布在整张图像上，以及带有可学习参数的卷积是一种用少量参数在多个位置上提取相似特征的有效方式。在那时候，没有 GPU 帮助训练，甚至 CPU 的速度也很慢。因此，能够保存参数以及计算过程是一个关键进展。这和将每个像素用作一个大型多层神经网络的单独输入相反。LeNet5 阐述了那些像素不应该被使用在第一层，因为图像具有很强的空间相关性，而使用图像中独立的像素作为不同的输入特征则利用不到这些相关性。

LeNet [2] 作为一个早期用来识别手写数字图像的卷积神经网络，展示了通过`梯度下降`训练卷积神经网络可以达到手写数字识别在当时最先进的结果。如下图所示：


![LeNet模型](https://d2l.ai/_images/lenet.svg)

LeNet的模型结构分为`卷积层块`和`全连接层块`两个部分：

- 卷积层块`保留输入形状`，使图像的像素在高和宽两个方向上的相关性均可能被有效识别，并且通过`滑动窗口`将同一卷积核与不同位置的输入重复计算，从而避免`参数尺寸过大`。
	- `卷积层块`里的基本单位是`卷积层`后接`平均池化层`：`卷积层`用来`识别图像里的空间模式`，之后的`平均池化层`则用来`降低卷积层对位置的敏感性`。`卷积层块`由`卷积层+平均池化层`重复堆叠构成。在卷积层块中，每个卷积层都使用$5 \times 5$的窗口，并在输出上使用Sigmoid激活函数。`第一个卷积层`输出通道数为6，`第二个卷积层`输出通道数则增加到16。

- 全连接层块`将卷积层块的输出中每个样本变平`（Flatten），即`输入形状将变成二维`，其中第一维是`小批量中的样本（batch_size）`，第二维是`每个样本变平后的向量表示`，从而进行分类。
	- `全连接层块`含`3个全连接层`。它们的输出个数分别是120、84和10，其中10为输出的类别个数。

![Image Name](https://cdn.kesci.com/upload/image/q5ndxi6jl5.png?imageView2/0/w/640/h/640)

![LeNet-5](https://cdn.kesci.com/upload/image/q6ey226gzx.jpeg?imageView2/0/w/960/h/960)
**LeNet-5结构：LeNet-5包含七层，不包括输入，每一层都包含可训练参数（权重）。使用的输入数据是32 * 32像素的图像。**

**以下是各层结构的介绍，其中卷积层将用Cx表示，子采样层则被标记为Sx，完全连接层被标记为Fx，x是层索引。**

- **C1层**是具有六个5 x 5的卷积核的卷积层（convolution），特征映射的大小为28 * 28，这样可以防止输入图像的信息掉出卷积核边界。C1包含156个可训练参数和122304个连接。

- **S2层**是输出6个大小为14 x 14的特征图的子采样层（subsampling/pooling）。每个特征地图中的每个单元连接到C1中的对应特征地图中的2 x 2个邻域。S2中单位的四个输入相加，然后乘以可训练系数（权重），然后加到可训练偏差（bias）。结果通过S形函数传递。由于2 x 2个感受域不重叠，因此S2中的特征图只有C1中的特征图的一半行数和列数。S2层有12个可训练参数和5880个连接。

- **C3层**是具有16个5 x 5的卷积核的卷积层。前六个C3特征图的输入是S2中的三个特征图的每个连续子集，接下来的六个特征图的输入则来自四个连续子集的输入，接下来的三个特征图的输入来自不连续的四个子集。最后，最后一个特征图的输入来自S2所有特征图。C3层有1516个可训练参数和156000个连接。

- **S4层**是与S2类似，大小为 2 x 2 ，输出为16个 5 x 5 的特征图。S4层有32个可训练参数和2000个连接。

- **C5层**是具有120个大小为5 x 5的卷积核的卷积层。每个单元连接到S4的所有16个特征图上的5 x 5邻域。这里，因为S4的特征图大小也是5 x 5，所以C5的输出大小是1 x 1。因此S4和C5之间是完全连接的。C5被标记为卷积层，而不是完全连接的层，是因为如果LeNet-5输入变得更大而其结构保持不变，则其输出大小会大于1 x 1，即不是完全连接的层了。C5层有48120个可训练连接。

- **F6层**完全连接到C5，输出84张特征图。它有10164个可训练参数。这里84与输出层的设计有关。

## AlexNet

2012年，AlexNet [3] 横空出世，使用了`8层卷积神经网络`，并以很大的优势赢得了ImageNet 2012图像识别挑战赛。

AlexNet与LeNet的设计理念非常相似，但相对较小的LeNet相比，AlexNet包含`5层卷积`和`2层全连接隐藏层`，以及`1个全连接输出层`，`模型参数`也大大增加。由于早期`显存`的限制，最早的AlexNet使用`双数据流`的设计，使一个GPU只需要处理一半模型。如下图所示：

![AlexNet模型](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter05/5.6_alexnet.png)
上图中的卷积部分都是画成上下两块，意思是说吧这一层计算出来的feature map分开，但是前一层用到的数据要看连接的虚线，如图中input层之后的第一层第二层之间的虚线是分开的，是说二层上面的128map是由一层上面的48map计算的，下面同理；而第三层前面的虚线是完全交叉的，就是说每一个192map都是由前面的128+128=256map同时计算得到的。

AlexNet首次证明了神经网络以端到端（end-to-end）的方式学习到的特征可以超越手工设计的特征，从而一举打破计算机视觉研究的前状。

AlexNet首次证明了学习到的特征可以超越人工设计的特征，从而一举打破计算机视觉研究的前状。   
**特征：**
1. 8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层。
2. 使用修正的线性单元（ReLU）作为非线性：将`Sigmoid`激活函数改成了更加简单的`ReLU`激活函数。
3. 在训练的时候使用 Dropout 技术有选择地忽视单个神经元，以避免模型过拟合：用`Dropout`来控制全连接层的模型复杂度。
4. 覆盖进行最大池化，避免平均池化的平均化效果。
5. 引入`数据增强`，如翻转、裁剪和颜色变化，从而进一步扩大数据集来缓解过拟合。

![Image Name](https://cdn.kesci.com/upload/image/q5kv4gpx88.png?imageView2/0/w/640/h/640)

- Lenet & AlexNet （模型介绍）
- Lenet & AlexNet （网络搭建）
- 运用Lenet & AlexNet进行图像识别Fashion-MNIST数据集（具体实现）

**AlexNet网络结构：（多层不同大小的卷积层Conv+全连接Fc）**
![AlexNet](https://cdn.kesci.com/upload/image/q6gbax7m8o.jpg?imageView2/0/w/960/h/960)
- **Conv1 Step:**
> 输入数据：227×227×3
卷积核：11×11×3；步长：4；数量（也就是输出个数）：96
卷积后数据：55×55×96 （原图N×N，卷积核大小n×n，卷积步长大于1为k，输出维度是(N-n)/k+1）
relu1后的数据：55×55×96
Max pool1的核：3×3，步长：2
Max pool1后的数据：27×27×96
norm1：local_size=5 （LRN(Local Response Normalization） 局部响应归一化）
最后的输出：27×27×96
- **Conv2 Step:**
> 输入数据：27×27×96
卷积核：5×5；步长：1；数量（也就是输出个数）：256
卷积后数据：27×27×256 （做了Same padding（相同补白），使得卷积后图像大小不变。）
relu2后的数据：27×27×256
Max pool2的核：3×3，步长：2
Max pool2后的数据：13×13×256 （（27-3）/2+1=13 ）
norm2：local_size=5 （LRN(Local Response Normalization） 局部响应归一化）
最后的输出：13×13×256
conv2中使用了same padding，保持了卷积后图像的宽高不缩小。
- **Conv3 Step:**
> 输入数据：13×13×256
卷积核：3×3；步长：1；数量（也就是输出个数）：384
卷积后数据：13×13×384 （做了Same padding（相同补白），使得卷积后图像大小不变。）
relu3后的数据：13×13×384
最后的输出：13×13×384
conv3层没有Max pool层和norm层
- **Conv4 Step:**
> 输入数据：13×13×384
卷积核：3×3；步长：1；数量（也就是输出个数）：384
卷积后数据：13×13×384 （做了Same padding（相同补白），使得卷积后图像大小不变。）
relu4后的数据：13×13×384
最后的输出：13×13×384
conv4层也没有Max pool层和norm层
- **Conv5 Step:**
> 输入数据：13×13×384
卷积核：3×3；步长：1；数量（也就是输出个数）：256
卷积后数据：13×13×256 （做了Same padding（相同补白），使得卷积后图像大小不变。）
relu5后的数据：13×13×256
Max pool5的核：3×3，步长：2
Max pool2后的数据：6×6×256 （（13-3）/2+1=6 ）
最后的输出：6×6×256
conv5层有Max pool，没有norm层
- **Fc6 Step:**
> 输入数据：6×6×256
全连接输出：4096×1
relu6后的数据：4096×1
drop out6后数据：4096×1
最后的输出：4096×1
- **Fc7 Step:**
> 输入数据：4096×1
全连接输出：4096×1
relu7后的数据：4096×1
drop out7后数据：4096×1
最后的输出：4096×1
- **Fc8 Step:**
> 输入数据：4096×1
全连接输出：1000
fc8输出一千种分类的概率

 **整体来看，AlexNet的卷积核从11到5再到3不断变小，而feature map也通过重叠式max pool在第1、2、5层折半式缩小，到第5个卷积层后，图像特征已经提炼得足够充分，便用两个全连接层和一个softmax层组合得出最终的分类结果。**

### 问题一：
- 阅读提出上述两种网络（**LeNet和AlexNet**）的相关论文，试从数据集的**预处理**、**激活函数**的使用、**训练方法**的改进以及**模型结构**的变化等角度，从理论层面分析比较`LeNet与AlexNet的结构差异`，并尝试解释AlexNet为什么会具有对计算机视觉任务优越的处理性能。

- AlexNet对Fashion-MNIST数据集来说可能过于复杂，请尝试对**模型**进行**简化**来使训练更快，同时保证`分类准确率`（accuracy）不明显下降（不低于85%），并将**简化后的结构**、**节省的训练时间**以及**下降的准确率**等相关指标以`表格`的形式进行总结分析。

**从理论层面分析比较`LeNet与AlexNet的结构差异`**

- AlexNet采用了Relu激活函数：ReLU(x) = max(x,0)

- AlexNet另一个创新是LRN(Local Response Normalization） 局部响应归一化，LRN模拟神经生物学上一个叫做 侧抑制（lateral inhibitio）的功能，侧抑制指的是被激活的神经元会抑制相邻的神经元。LRN局部响应归一化借鉴侧抑制的思想实现局部抑制，使得响应比较大的值相对更大，提高了模型的泛化能力。LRN只对数据相邻区域做归一化处理，不改变数据的大小和维度。

- AlexNet还应用了Overlapping（重叠池化），重叠池化就是池化操作在部分像素上有重合。池化核大小是n×n，步长是k，如果k=n，则是正常池化，如果 k<n, 则是重叠池化。官方文档中说明，重叠池化的运用减少了top-5和top-1错误率的0.4%和0.3%。重叠池化有避免过拟合的作用。

- AlexNet在fc6、fc7全连接层引入了drop out的功能。dropout是指在深度学习网络的训练过程中，对于神经网络单元，按照一定的概率（AlexNet是50%，这种情况下随机生成的网络结构最多）将其暂时从网络中丢弃（保留其权值），不再对前向和反向传输的数据响应。注意是暂时，对于随机梯度下降来说，由于是随机丢弃，故而相当于每一个mini-batch都在训练不同的网络，drop out可以有效防止模型过拟合，让网络泛化能力更强，同时由于减少了网络复杂度，加快了运算速度。还有一种观点认为drop out有效的原因是对样本增加来噪声，变相增加了训练样本。

- 数据增强：在数据处理这部分作者提到过将每张图片处理为256××256的大小，但网络结构图中的输入却为224××224，这是因为作者在256××256大小的图片上使用了一个224××224的滑动窗口，将每个滑动窗口中的内容作为输入，这样就能将整个数据集扩大到原来的(256−224)×(256−224)=1024(256−224)×(256−224)=1024倍



**LeNet**

In [ ]:
import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l
import torch
import torch.nn as nn
import torch.optim as optim
import time

In [ ]:
# 网络结构 net
class Flatten(torch.nn.Module):  # 展平操作
    def forward(self, x):
        return x.view(x.shape[0], -1) 

class Reshape(torch.nn.Module): # 将图像大小重定型
    def forward(self, x):
        return x.view(-1,1,28,28)      # (B x C x H x W) 批量尺寸*通道数*高*宽
    
net = torch.nn.Sequential(     # Lelet                                                  
    # 重定型
    Reshape(), 
    
    # 卷积层 [（nh - kh + ph + sh）/ sh ] * [（nw - kw + pw + sw）/ sw ]向下取整
    nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2), # b*1*28*28  => b*6*28*28
    # sigmoid激活函数
    nn.Sigmoid(),                                                       
    # 平均池化层
    nn.AvgPool2d(kernel_size=2, stride=2), # b*6*28*28  => b*6*14*14
    
    # 卷积层
    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5), # b*6*14*14  => b*16*10*10
    # sigmoid激活函数
    nn.Sigmoid(),
    # 平均池化层
    nn.AvgPool2d(kernel_size=2, stride=2),  # b*16*10*10 => b*16*5*5
    
    # 扁平操作
    Flatten(),  # b*16*5*5   => b*400
    
    # 全连接操作
    nn.Linear(in_features=16*5*5, out_features=120),
    nn.Sigmoid(),
   
    # 全连接操作
    nn.Linear(120, 84),
    nn.Sigmoid(),
    
    # 全连接操作
    nn.Linear(84, 10)
)

In [ ]:
# 构造一个高和宽均为28的单通道数据样本，并逐层进行前向计算来查看每个层的输出形状
X = torch.randn(size=(1,1,28,28), dtype = torch.float32) # 1*1*28*28
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

In [ ]:
# 划分数据集
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(
    batch_size=batch_size, root='/home/kesci/input/FashionMNIST2065')
print(len(train_iter)) # 打印训练集中的批次数，每个批次含有样本数（图片数）为batch_size（256）

In [ ]:
# 数据展示
import matplotlib.pyplot as plt
def show_fashion_mnist(images, labels):
    d2l.use_svg_display()
    # 这里的_表示我们忽略（不使用）的变量
    _, figs = plt.subplots(1, len(images), figsize=(12, 12))
    for f, img, lbl in zip(figs, images, labels):
        f.imshow(img.view((28, 28)).numpy())
        f.set_title(lbl)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)
    plt.show()

for Xdata,ylabel in train_iter:
    break
X, y = [], []
for i in range(10):
    print(Xdata[i].shape,ylabel[i].numpy())
    X.append(Xdata[i]) # 将第i个feature加到X中
    y.append(ylabel[i].numpy()) # 将第i个label加到y中
show_fashion_mnist(X, y)

In [ ]:
# This function has been saved in the d2l package for future use
# use GPU
def try_gpu():
    """If GPU is available, return torch.device as cuda:0; else return torch.device as cpu."""
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

device = try_gpu()
device

In [ ]:
# 计算准确率
'''
(1). net.train()
  启用 BatchNormalization 和 Dropout，将BatchNormalization和Dropout置为True
(2). net.eval()
不启用 BatchNormalization 和 Dropout，将BatchNormalization和Dropout置为False
'''
def evaluate_accuracy(data_iter,net,device=torch.device('cpu')):
    """Evaluate accuracy of a model on the given data set."""
    acc_sum,n = torch.tensor([0],dtype=torch.float32,device=device),0
    for X,y in data_iter:
        # If device is the GPU, copy the data to the GPU.
        X,y = X.to(device),y.to(device)
        net.eval()
        with torch.no_grad():
            y = y.long() # 数据类型long
            # [[0.2,0.4,0.5,0.6,0.8],[ 0.1,0.2,0.4,0.3,0.1]] => [4,2]
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))  
            n += y.shape[0]
    return acc_sum.item()/n

In [ ]:
# 定义训练函数
def train_ch5(net, train_iter, test_iter,criterion, num_epochs, batch_size, device,lr=None):
    """Train and evaluate a model with CPU or GPU."""
    print('training on', device)
    net.to(device)
    optimizer = optim.SGD(net.parameters(), lr=lr) # SGD随机梯度下降 ADAM
    for epoch in range(num_epochs):
        train_l_sum = torch.tensor([0.0],dtype=torch.float32,device=device)
        train_acc_sum = torch.tensor([0.0],dtype=torch.float32,device=device)
        n, start = 0, time.time()
        for X, y in train_iter:
            net.train()
            
            optimizer.zero_grad()
            X,y = X.to(device),y.to(device) 
            y_hat = net(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                y = y.long()
                train_l_sum += loss.float()
                train_acc_sum += (torch.sum((torch.argmax(y_hat, dim=1) == y))).float()
                n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net,device)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum/n, train_acc_sum/n, test_acc,
                 time.time() - start))

In [ ]:
# 训练
lr, num_epochs = 0.9, 10

def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

net.apply(init_weights)
net = net.to(device)

# 交叉熵描述了两个概率分布之间的距离，交叉熵越小说明两者之间越接近
criterion = nn.CrossEntropyLoss()
train_ch5(net, train_iter, test_iter, criterion,num_epochs, batch_size,device, lr)

In [ ]:
# 测试
for testdata,testlabe in test_iter:
    testdata,testlabe = testdata.to(device),testlabe.to(device)
    break
print(testdata.shape,testlabe.shape)
net.eval()
y_pre = net(testdata)
print(torch.argmax(y_pre,dim=1)[:10])
print(testlabe[:10])

**AlexNet**

In [ ]:
# Kaggle 代码：https://www.kaggle.com/boyuai/boyu-d2l-modernconvolutionalnetwork

import time
import torch
from torch import nn, optim
import torchvision
import numpy as np
import sys
sys.path.append("/home/kesci/input/") 
import d2lzh1981 as d2l
import os
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding默认为0
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
            
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            # 前两个卷积层后不使用池化层来减小输入的高和宽
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
         # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃Dropout层来缓解过拟合
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096), # 通道数*宽*高
            nn.ReLU(),
            nn.Dropout(0.5),
            # 由于使用CPU镜像，精简网络，若为GPU镜像可添加该层
            # nn.Linear(4096, 4096),
            # nn.ReLU(),
            # nn.Dropout(0.5),

            # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 10),
        )

    def forward(self, img): # 对卷积层块和全连接层块进行连接concat，需要进行reshape操作
                            # conv的输出是一个四维张量 B(batch_size) * C(channels) * H(height) * W(width)
                            # fc的输出是一个二维张量 Bacth_size * Hidden_numbers
        feature = self.conv(img)
        # 将feature通过view操作转换为2维张量，第一维为batch_size，第二维自动计算 -1等价于256*5*5
        output = self.fc(feature.view(img.shape[0], -1)) # view操作类似于reshape操作
        return output # batch_size * 10

In [ ]:
net = AlexNet()
print(net)

In [ ]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def load_data_fashion_mnist(batch_size, resize=None, root='/home/kesci/input/FashionMNIST2065'):
    """Download the fashion mnist dataset and then load into memory."""
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)

    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_iter, test_iter

# batchsize = 128
batch_size = 16
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size,224)
for X, Y in train_iter:
    print('X =', X.shape,
        '\nY =', Y.type(torch.int32))
    break
    

In [ ]:
lr, num_epochs = 0.001, 3
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

## VGG

AlexNet在LeNet的基础上增加了3个卷积层，同时对网络的卷积窗口、输出通道数和构造顺序均做了大量的调整。AlexNet指明了深度卷积神经网络可以取得出色的结果，基于这一个理念，牛津大学的实验室Visual Geometry Group实验室提出了VGG [4] 网络，提供了通过重复使用简单的基础块来构建深度模型的思路。VGG每个基础块组成规律是：连续使用数个相同的填充为1、窗口形状为$3\times 3$的卷积层后接上一个步幅为2、窗口形状为$2\times 2$的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。如下图所示：

![VGG模型](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/vgg.jpg)

可以看到，每次经过基础块以后，网络会将输入的高和宽减半，直到最终高和宽变成7后传入全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口大小一样，VGG这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

VGG：通过重复使用简单的`基础块`来构建深度模型。  
Block:数个相同的填充为1、窗口形状为$3\times 3$的卷积层（使得图片输出的宽和高不变）,接上一个步幅为2、窗口形状为$2\times 2$的最大池化层（使得图片输出的宽和高减半）。  
卷积层保持输入的高和宽不变，而池化层则对其减半。


![Image Name](https://cdn.kesci.com/upload/image/q5l6vut7h1.png?imageView2/0/w/640/h/640)

### 问题二：
- **LeNet与AlexNet**在`接近图像输入的卷积模块`中都引入了**较大尺寸的卷积核**，如$5\times 5$或者$7\times 7$的卷积窗口来捕捉更大范围的图像信息，试分析**VGG每个基础块**的固定设计是否会影响到**图像的粗粒度**信息提取，并且对比**不同结构模块输出的特征图**进行对比。
- 尝试将Fashion-MNIST中图像的`高和宽由224改为96`，试分析**VGG网络**的参数变化情况，并且对比模型训练时间、分类准确率（accuracy）等实验指标受到的影响，以表格的形式进行总结分析。

**VGG（多层（更多）同样大小的卷积层+全连接层）:**

![VGG](https://cdn.kesci.com/upload/image/q6gd1zokat.jpg?imageView2/0/w/960/h/960)


**VGG较于AlexNet改进后的特点：**

- 去掉了LRN层，作者发现深度网络中LRN的作用并不明显，干脆取消了

- 采用更小的卷积核-3x3，Alexnet中使用了更大的卷积核，比如有7x7的，因此VGG相对于Alexnet而言，参数量更少

- 池化核变小，VGG中的池化核是2x2，stride为2，Alexnet池化核是3x3，步长为2

VGG可以看做**在神经网络中越高层的网络应该越稀疏**，同时表现力更强。这样可以**避免过深的神经网络计算量过大**，容易`过拟合`的问题。

**VGG**

In [ ]:
def vgg_block(num_convs, in_channels, out_channels): #卷积层个数，输入通道数，输出通道数
    blk = []
    for i in range(num_convs):
        if i == 0:
            blk.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        else:
            blk.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        blk.append(nn.ReLU())
    blk.append(nn.MaxPool2d(kernel_size=2, stride=2)) # 这里会使宽高减半
    return nn.Sequential(*blk)

In [ ]:
conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))
# 经过5个vgg_block, 宽高会减半5次, 变成 224/32 = 7
fc_features = 512 * 7 * 7 # c * w * h
fc_hidden_units = 4096 # 任意

In [ ]:
def vgg(conv_arch, fc_features, fc_hidden_units=4096):
    net = nn.Sequential()
    # 卷积层部分
    for i, (num_convs, in_channels, out_channels) in enumerate(conv_arch):
        # 每经过一个vgg_block都会使宽高减半
        net.add_module("vgg_block_" + str(i+1), vgg_block(num_convs, in_channels, out_channels))
    # 全连接层部分
    net.add_module("fc", nn.Sequential(d2l.FlattenLayer(),
                                 nn.Linear(fc_features, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, 10)
                                ))
    return net

In [ ]:
net = vgg(conv_arch, fc_features, fc_hidden_units)
X = torch.rand(1, 1, 224, 224) # N C H W

# named_children获取一级子模块及其名字(named_modules会返回所有子模块,包括子模块的子模块)
for name, blk in net.named_children(): 
    X = blk(X)
    print(name, 'output shape: ', X.shape)

In [ ]:
ratio = 8
small_conv_arch = [(1, 1, 64//ratio), (1, 64//ratio, 128//ratio), (2, 128//ratio, 256//ratio), 
                   (2, 256//ratio, 512//ratio), (2, 512//ratio, 512//ratio)]
net = vgg(small_conv_arch, fc_features // ratio, fc_hidden_units // ratio)
print(net)

In [ ]:
batchsize = 16
# batch_size = 64
# 如出现“out of memory”的报错信息，可减小batch_size或resize
# train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

## NiN

之前介绍的LeNet、AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，AlexNet和VGG对LeNet的改进主要在于如何对这两个模块加宽（增加通道数）和加深。网络中的网络（NiN）[5] 提出了另外一个思路，即串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络。

卷积层的输入和输出通常是四维数组（样本，通道，高，宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维，$1\times 1$卷积层可以看成全连接层，其中空间维度（高和宽）上的每个元素相当于样本，通道相当于特征。因此，NiN使用$1\times 1$卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。下图对比了NiN同AlexNet和VGG等网络在结构上的主要区别。

![左图是AlexNet和VGG的网络结构局部，右图是NiN的网络结构局部](http://zh.d2l.ai/_images/nin.svg)

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的$1\times 1$卷积层串联而成。其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。


LeNet、AlexNet和VGG：先以由`卷积层`构成的模块充分抽取`空间特征`，再以由`全连接层`构成的模块来输出`分类`结果。  
NiN：`串联`多个由`卷积层`和`全连接层`构成的`小网络`来构建多个深层网络。  
`输出通道数`等于`标签类别数`的NiN块，然后使用`全局平均池化层`对每个通道中所有元素求平均并直接用于分类。  

![Image Name](https://cdn.kesci.com/upload/image/q5l6u1p5vy.png?imageView2/0/w/960/h/960)

**1×1卷积核**作用   
1.放缩通道数：通过控制卷积核的数量达到通道数的放缩。  
2.增加非线性：1×1卷积核的卷积过程相当于全连接层的计算过程，并且还加入了非线性激活函数，从而可以增加网络的非线性。  
3.计算参数少   

## GoogLeNet

在2014年的ImageNet图像识别挑战赛中，一个名叫GoogLeNet的网络结构大放异彩 [6] ，它虽然在名字上向LeNet致敬，但在网络结构上已经很难看到LeNet的影子。GoogLeNet吸收了NiN中网络串联网络的思想，并在此基础上做了很大改进。GoogLeNet中的基础卷积块叫作Inception块，得名于同名电影《盗梦空间》（Inception）。与上NiN块相比，这个基础块在结构上更加复杂，如下图所示：

![Inception块的结构](http://zh.d2l.ai/_images/inception.svg)

Inception块里有4条并行的线路。前3条线路使用窗口大小分别是$1\times 1$、$3\times 3$和$5\times 5$的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做$1\times 1$卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用$3\times 3$最大池化层，后接$1\times 1$卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后将每条线路的输出在通道维上连结，并输入接下来的层中去。Inception块中可以自定义的超参数是每个层的输出通道数，以此来控制模型复杂度。

GoogLeNet中1*1的卷积层与 NiN中1*1的卷积层的区别？？？
1. 由Inception基础块组成。  
2. Inception块相当于多个有4条线路的网络。它通过不同窗口形状的卷积层和最大池化层来合并抽取信息，并使用1×1卷积层减少通道数从而降低模型复杂度。   
3. 可以定义的超参数是每个层的输出通道数，我们以此来控制模型复杂度。 

![Image Name](https://cdn.kesci.com/upload/image/q5l6uortw.png?imageView2/0/w/640/h/640)

### 问题三：
- 对比**AlexNet、VGG和NiN、GoogLeNet**的`模型参数尺寸`，从理论的层面分析为什么后两个网络可以显著减小模型参数尺寸？
- GoogLeNet有数个后续版本，包括加入**批量归一化层 [7]**、对**Inception块做调整 [8]** 和**加入残差连接 [9]**，请尝试实现并运行它们，然后观察实验结果，以表格的形式进行总结分析。

**NiN（卷积层+用1X1层替代全连接层）**

**NIN结构（右边）与AlexNet、VGG（左边）的区别**

![NiN](http://zh.d2l.ai/_images/nin.svg)


> **VGG、AlexNet的绝大多数参数都集中于最后几个全连接层上，全连接层的作用在于`线性强`，`参数多`，还容易`过拟合`**。

> **NIN便使用`1x1的卷积层`很好地解决了以上问题，利用多个`普通卷积层 + 1 * 1的卷积层`的嵌套，不仅可以达到良好的效果，而且大大降低了参数。**

In [ ]:
def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        nn.ReLU(),
                        
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk

In [ ]:
# 已保存在d2lzh_pytorch
class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])

net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, stride=4, padding=0), # 0
    nn.MaxPool2d(kernel_size=3, stride=2),
    
    nin_block(96, 256, kernel_size=5, stride=1, padding=2), # 2 图中有误
    nn.MaxPool2d(kernel_size=3, stride=2),
    
    nin_block(256, 384, kernel_size=3, stride=1, padding=1), # 1
    nn.MaxPool2d(kernel_size=3, stride=2), 
    
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, stride=1, padding=1), # 1
    GlobalAvgPool2d(), 
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
    d2l.FlattenLayer())

In [ ]:
X = torch.rand(1, 1, 224, 224) # N C H W
for name, blk in net.named_children(): 
    X = blk(X)
    print(name, 'output shape: ', X.shape)

In [ ]:
batch_size = 128
# 如出现“out of memory”的报错信息，可减小batch_size或resize
#train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

lr, num_epochs = 0.002, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

**Inception（一分四，然后做一些不同大小的卷积，之后再堆叠feature map）**

![Inception](https://cdn.kesci.com/upload/image/q6gclum6jd.jpg?imageView2/0/w/960/h/960)

> 可以看出这个结构很大程度上借鉴了NIN的思想，大量使用1x1的卷积层，同时也有创新，一个inception同时使用多个不同尺寸的卷积层，以一种结构化的方式来捕捉不同尺寸的信息，很大程度地降低了参数量和计算量。

**GoogleNet（多个Inception的叠加：）**

![GoogleNet](https://cdn.kesci.com/upload/image/q6gcmvksrd.jpg?imageView2/0/w/960/h/960)


> **GoogLeNet可理解为：进入第一个inception前，feature map为 56x56，经过两个inception后，缩小为28x28，经过7个inception后变成14x14，经过9个inception后为7x7。最后通过7x7的average pool变成1x1。**

In [ ]:
class Inception(nn.Module):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1)  # 在通道维上连结输出

In [ ]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   d2l.GlobalAvgPool2d())

net = nn.Sequential(b1, b2, b3, b4, b5, 
                    d2l.FlattenLayer(), nn.Linear(1024, 10))

net = nn.Sequential(b1, b2, b3, b4, b5, d2l.FlattenLayer(), nn.Linear(1024, 10))

X = torch.rand(1, 1, 96, 96) # N C H W

for blk in net.children(): 
    X = blk(X)
    print('output shape: ', X.shape)

# batchsize = 128
batch_size = 16
# 如出现“out of memory”的报错信息，可减小batch_size或resize
# train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

## 参考文献
[1] Xiao, H., Rasul, K., & Vollgraf, R. (2017). Fashion-mnist: a novel image dataset for benchmarking machine learning algorithms. arXiv preprint arXiv:1708.07747.

[2] LeCun, Y., Bottou, L., Bengio, Y., & Haffner, P. (1998). Gradient-based learning applied to document recognition. Proceedings of the IEEE, 86(11), 2278-2324.

[3] Krizhevsky, A., Sutskever, I., & Hinton, G. E. (2012). Imagenet classification with deep convolutional neural networks. In Advances in neural information processing systems (pp. 1097-1105).

[4] Simonyan, K., & Zisserman, A. (2014). Very deep convolutional networks for large-scale image recognition. arXiv preprint arXiv:1409.1556.

[5] Lin, M., Chen, Q., & Yan, S. (2013). Network in network. arXiv preprint arXiv:1312.4400.

[6] Szegedy, C., Liu, W., Jia, Y., Sermanet, P., Reed, S., & Anguelov, D. & Rabinovich, A.(2015). Going deeper with convolutions. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1-9).

[7] Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training by reducing internal covariate shift. arXiv preprint arXiv:1502.03167.

[8] Szegedy, C., Vanhoucke, V., Ioffe, S., Shlens, J., & Wojna, Z. (2016). Rethinking the inception architecture for computer vision. In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition (pp. 2818-2826).

[9] Szegedy, C., Ioffe, S., Vanhoucke, V., & Alemi, A. A. (2017, February). Inception-v4, inception-resnet and the impact of residual connections on learning. In Proceedings of the AAAI Conference on Artificial Intelligence (Vol. 4, p. 12).

## 项目报告
本次大作业的终审评估以项目报告作为重要依据，开放题报告的内容和排版要求请下载文件：


[termproject2.zip](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/termproject2.zip)

需要注意的是，文件中：
- `termproject.pdf`提供了项目报告的内容格式要求
- `termproject_exp.pdf`提供了项目报告的内容排版样例


推荐使用`LaTeX`软件进行报告的撰写，相关`.tex`以及`.sty`源文件一并附于文件夹中。
